# Introduction
The aim of this project is to conduct a sentiment analysis using python and twitter's API. The topic in my case is Expo 2020.

**What is Expo 2020?**
> Initiated in London 1851. It is a global gathering aimed to find solutions to challenges imposed by the current times. Aims  to create enriching and immersive experience. World expo traverses through different cities each time. It also revolves around certain themes. Currently, World expo is taking place in Dubai, UAE. Between Oct, 2021 and Mar, 2022. For more information please visit [expo2020dubai](https://www.expo2020dubai.com/)

**The goal of this project**
> As discussed above the main goal is to conduct a sentiment analysis and learn the basics of data science and big data projects. Since expo 2020 is considered an educational exhibition that revolves around modern day problems and is hosted currently in the middle east. The aim is to measure the awareness of the arab society - *By arab society we mean anyone who posts their opinions in arabic*



This part is about preprocessing the data. we will preform quality assessments and handle any data problems that is found.

In [34]:
import pandas as pd
import numpy as np
import pyarabic.araby as araby
import re

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer # devide the words 
from collections import Counter #count the number of word for each list 
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.isri import ISRIStemmer
import nltk
nltk.download('punkt')
import time
from textblob import TextBlob


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dalal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dalal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [35]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.float_format', lambda x: '%.5f' % x)


C:\Users\dalal\AppData\Local\Temp/ipykernel_11060/1364264969.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


### 1. Read the Data 
--- 
It is required to first read the data and ensure that it has ben parsed correctly. the following code block will read and store the .csv file into a data frame that we will clean up accordingly.

In [36]:
dirty_tweets = pd.read_csv('tweets_updated.csv')
display(dirty_tweets.head())
display(dirty_tweets.tail())


,index,ID,Tweet,Timestamp,Likes,Retweets,Length
0,0,1497300372569739264,قبيل الاحتفال بـ #اليوم_الدولي_للمرأة.. تستعد كوكبة من الفنانات الاستثنائيات وفرق موسيقية بقيادات نسائية لإبهار زوار إكسبو 2020 دبي\n\n#عينك_على_إكسبو \n@expo2020dubai #Expo2020\nhttps://t.co/3yfbIstF6Q,2022-02-25 20:00:01+00:00,1,0,198
1,1,1497299670522994691,#JIMMY_4040 \nلحظات نزول دابانغ سلمان خان على أرض المسرح في اكسبو 2020 وهو يقف على مجسم لجناحي صقر والتصفيق مستمر من قبل الجمهور\n#dabanggtourreloaded #Expo2020 @expo2020dubai #Dubai https://t.co/gTBM7OL4Fv,2022-02-25 19:57:14+00:00,2,0,207
2,2,1497297255245754371,ليلة مميزة بإنتظارنا في @expo2020dubai يوم ٢٦ فبراير 😍\n\nألحانها شرقية 🎶 و نجومها إستثنائيين ✨\n\n#Watary | @ziadbourji | @josephattieh | @DalidaKhalil | #BashaarAlJawad | #Expo2020 | #Expo2020Dubai | #اكسبو2020 https://t.co/QtmvqEBOwZ,2022-02-25 19:47:38+00:00,0,0,232
3,3,1497295489481510914,@NancyAFashion أصدق اي حفلة احلى ولا اي فستان ؟؟؟\nهلأ اذا مستان انا بختارر #Expo2020 \nواذا حفلة فبختارر حفلة #Riyadh \nهو حفلات نانسي كلون حلوين يعني 😉💖😘💖😘💖😘\n@NancyAjram,2022-02-25 19:40:37+00:00,3,1,168
4,4,1497290589800480779,#JIMMY_4040 \nجنون ما بعده جنون دابانغ سلمان خان يستعرض أمام حشد كبير من معجبينه في اكسبو 2020\n#dabanggtourreloaded #Expo2020 @expo2020dubai #Dubaigirls https://t.co/5CSoLlsQW0,2022-02-25 19:21:09+00:00,5,2,178


,index,ID,Tweet,Timestamp,Likes,Retweets,Length
12470,12470,1509875727558189057,بعد توديع آخر ضيوف الحدث الدولي وإسدال الستار على فعالياته قام موظفو #إكسبو_2020 #دبي بالمشاركة في إغلاق البوابات الأربع في موقع الحدث الدولي\n\n@expo2020dubai\n#Expo2020 \nhttps://t.co/7YlXwPEegK,2022-04-01 12:50:00+00:00,0,0,192
12471,12471,1509875571114795011,جناح السعودية يختتم فعالياته في معرض “إكسبو 2020 دبي” بأرقام قياسية ونجاحات استثنائية https://t.co/EA7qSdmYuL,2022-04-01 12:49:22+00:00,0,0,109
12472,12472,1509875169782865932,#إكسبو_2020_دبي يختتم أعماله بأهم المؤتمرات العالمية.. ورحلة إلى #فلسطين في #إكسبو_2020\n\nالمزيد في حلقة جديدة من برنامج #إكسبو_في_أسبوع مع لين خضير\n\nhttps://t.co/jzjSXCQ06W https://t.co/Fu96hx1Q2y,2022-04-01 12:47:47+00:00,7,0,196
12473,12473,1509874863539998768,"نظمت الإدارة العامة لأمن المطارات ومبادرة ""الروح الايجابية"" في الإدارة العامة لإسعاد المجتمع في شرطة دبي، مسيرة ""رحلة الخمسين"" احتفاءً بمرور 50 عاماً على تأسيس الدولة، ونجاح فعاليات إكسبو 2020 دبي. https://t.co/y3EYrKi73V",2022-04-01 12:46:34+00:00,1,0,221
12474,12474,1509874781964980273,ختامها مسك..جناح #المملكة في #اكسبو_٢٠٢٠ #دبي يحقق أعلى نسب زيارات في تاريخ معارض إكسبو الدولية على الإطلاق بنحو 5 ملايين زائر من مختلف الجنسيات. https://t.co/3lUttJrODG,2022-04-01 12:46:14+00:00,1,0,169


### 1. Asses Duplicates
---
We will be assessing both duplicates and null vallues and based on our observation we will conduct the proper cleaning process
> We will find the duplicates of the tweets column only, since the other columns are gaurnteed to have duplicates and they are simply metadata. The function gets the duplicate rows without the first occurunce. we will then sum and check how much of our data is duplicated.

In [37]:
# Issue 1 - Duplicates
duplicate_tweets = dirty_tweets[dirty_tweets.duplicated(['Tweet'], keep='first')]
display(duplicate_tweets.tail())

print(f'the number of duplicates is {duplicate_tweets.shape[0]}')

,index,ID,Tweet,Timestamp,Likes,Retweets,Length
12453,12453,1509881814986412045,شُوفو زعِيم حَضنهآ كُيف خلآهآ 🇦🇪❤️\n\nاكسبو ٢٠٢٠، دبي\n#Expo2020 \n#Expo2020Dubai https://t.co/1ZXs6SISBL,2022-04-01 13:14:11+00:00,0,0,101
12454,12454,1509881803401801728,@HHShkMohd مبارك طويل العمر هذا الإنجاز الكبير #اكسبو_٢٠٢٠_دبي #محمد_بن_راشد_ال_مكتوم #فريق_اكسبو #ريم_الهاشمي #الامارات #Expo2020,2022-04-01 13:14:08+00:00,0,0,130
12457,12457,1509878788221513734,#الإمارات_تبتكر تكرم أفضل الابتكارات في الأجنحة في #اكسبو_٢٠٢٠_دبي #دبي @expo2020dubai @MBRinnovation \nhttps://t.co/mfQlIx9Uwt,2022-04-01 13:02:09+00:00,2,0,128
12464,12464,1509877148550586373,@alkhaleej @expo2020dubai انصصصصصدمت انه بس هالسنه وينتقل لدولة اخررر حسسسسسسافه 😢😢😩، قلت يمكن السنه الجاية انزل دبي واحضر #اكسبو_٢٠٢٠_دبي 😢,2022-04-01 12:55:39+00:00,0,0,140
12470,12470,1509875727558189057,بعد توديع آخر ضيوف الحدث الدولي وإسدال الستار على فعالياته قام موظفو #إكسبو_2020 #دبي بالمشاركة في إغلاق البوابات الأربع في موقع الحدث الدولي\n\n@expo2020dubai\n#Expo2020 \nhttps://t.co/7YlXwPEegK,2022-04-01 12:50:00+00:00,0,0,192


the number of duplicates is 4493


In [38]:
# Issue 1 - Fix
dirty_tweets = dirty_tweets.drop_duplicates(subset='Tweet', keep="first")
dirty_tweets = dirty_tweets.reset_index(drop=True)
dirty_tweets = dirty_tweets.drop(['index'], axis=1)
display(dirty_tweets.head())
display(dirty_tweets.tail())
print("Length", dirty_tweets.shape[0])


,ID,Tweet,Timestamp,Likes,Retweets,Length
0,1497300372569739264,قبيل الاحتفال بـ #اليوم_الدولي_للمرأة.. تستعد كوكبة من الفنانات الاستثنائيات وفرق موسيقية بقيادات نسائية لإبهار زوار إكسبو 2020 دبي\n\n#عينك_على_إكسبو \n@expo2020dubai #Expo2020\nhttps://t.co/3yfbIstF6Q,2022-02-25 20:00:01+00:00,1,0,198
1,1497299670522994691,#JIMMY_4040 \nلحظات نزول دابانغ سلمان خان على أرض المسرح في اكسبو 2020 وهو يقف على مجسم لجناحي صقر والتصفيق مستمر من قبل الجمهور\n#dabanggtourreloaded #Expo2020 @expo2020dubai #Dubai https://t.co/gTBM7OL4Fv,2022-02-25 19:57:14+00:00,2,0,207
2,1497297255245754371,ليلة مميزة بإنتظارنا في @expo2020dubai يوم ٢٦ فبراير 😍\n\nألحانها شرقية 🎶 و نجومها إستثنائيين ✨\n\n#Watary | @ziadbourji | @josephattieh | @DalidaKhalil | #BashaarAlJawad | #Expo2020 | #Expo2020Dubai | #اكسبو2020 https://t.co/QtmvqEBOwZ,2022-02-25 19:47:38+00:00,0,0,232
3,1497295489481510914,@NancyAFashion أصدق اي حفلة احلى ولا اي فستان ؟؟؟\nهلأ اذا مستان انا بختارر #Expo2020 \nواذا حفلة فبختارر حفلة #Riyadh \nهو حفلات نانسي كلون حلوين يعني 😉💖😘💖😘💖😘\n@NancyAjram,2022-02-25 19:40:37+00:00,3,1,168
4,1497290589800480779,#JIMMY_4040 \nجنون ما بعده جنون دابانغ سلمان خان يستعرض أمام حشد كبير من معجبينه في اكسبو 2020\n#dabanggtourreloaded #Expo2020 @expo2020dubai #Dubaigirls https://t.co/5CSoLlsQW0,2022-02-25 19:21:09+00:00,5,2,178


,ID,Tweet,Timestamp,Likes,Retweets,Length
7977,1509875797544378374,عمان المجد والتاريخ\nوالتقدم العلمي وحصول\nالسلطنة على الجائزة الذهبية\nفي أكسبو 2020 دبي . https://t.co/tu9MZhuGpP,2022-04-01 12:50:16+00:00,0,0,112
7978,1509875571114795011,جناح السعودية يختتم فعالياته في معرض “إكسبو 2020 دبي” بأرقام قياسية ونجاحات استثنائية https://t.co/EA7qSdmYuL,2022-04-01 12:49:22+00:00,0,0,109
7979,1509875169782865932,#إكسبو_2020_دبي يختتم أعماله بأهم المؤتمرات العالمية.. ورحلة إلى #فلسطين في #إكسبو_2020\n\nالمزيد في حلقة جديدة من برنامج #إكسبو_في_أسبوع مع لين خضير\n\nhttps://t.co/jzjSXCQ06W https://t.co/Fu96hx1Q2y,2022-04-01 12:47:47+00:00,7,0,196
7980,1509874863539998768,"نظمت الإدارة العامة لأمن المطارات ومبادرة ""الروح الايجابية"" في الإدارة العامة لإسعاد المجتمع في شرطة دبي، مسيرة ""رحلة الخمسين"" احتفاءً بمرور 50 عاماً على تأسيس الدولة، ونجاح فعاليات إكسبو 2020 دبي. https://t.co/y3EYrKi73V",2022-04-01 12:46:34+00:00,1,0,221
7981,1509874781964980273,ختامها مسك..جناح #المملكة في #اكسبو_٢٠٢٠ #دبي يحقق أعلى نسب زيارات في تاريخ معارض إكسبو الدولية على الإطلاق بنحو 5 ملايين زائر من مختلف الجنسيات. https://t.co/3lUttJrODG,2022-04-01 12:46:14+00:00,1,0,169


Length 7982


In [39]:
# Issue 2 - Fix
def remove_diacritics(tweet):
    tweet = araby.strip_diacritics(tweet)
    tweet = araby.strip_tashkeel(tweet)
    tweet = araby.normalize_alef(tweet)
    tweet = araby.normalize_hamza(tweet)
    tweet = araby.normalize_ligature(tweet)

    # remove repeated characters 
    tweet = re.sub(r'(.)\1+', r'\1', tweet)
    return tweet


In [52]:
# Issue 3 - fix
def remove_emojis_unrelated_chars(tweet):
    # replace @username with empty string
    tweet = re.sub('@[^\s]+', ' ', tweet)
    
    tweet = re.sub('[/\W+/g]', ' ',  tweet)
    # remove underscore
    tweet = re.sub('[_]', ' ',  tweet)
    return tweet


In [53]:
# Issue 5 - fix
def remove_links_foreign_words(tweet):
    # convert www.* or https?://* to " "
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', ' ', tweet)

    # remove english words
    tweet = re.sub(r'\s*[A-Za-z]+\b', ' ', tweet).rstrip()

    # remove new lines with white space
    tweet = tweet.replace('\n', ' ')
    tweet = tweet.replace('\t', ' ')

    return tweet


In [54]:
# Issue 6 - fix
ar_stops = set(stopwords.words('arabic'))
#remove stop_words
stop_words = {"،","","ورحمه","وبركاته","عليكم","السلام","آض","آمينَ","آه","آهاً","آي","أ","أب","أجل","أجمع","أخ","أخذ","أصبح","أضحى","أقبل","أقل","أكثر","ألا","أم","أما","أمامك","أمامكَ","أمسى","أمّا","أن","أنا","أنت","أنتم","أنتما","أنتن","أنتِ","أنشأ","أنّى","أو","أوشك","أولئك","أولئكم","أولاء","أولالك","أوّهْ","أي","أيا","أين","أينما","أيّ","أَنَّ","أََيُّ","أُفٍّ","إذ","إذا","إذاً","إذما","إذن","إلى","إليكم","إليكما","إليكنّ","إليكَ","إلَيْكَ","إلّا","إمّا","إن","إنّما","إي","إياك","إياكم","إياكما","إياكن","إيانا","إياه","إياها","إياهم","إياهما","إياهن","إياي","إيهٍ","إِنَّ","ا","ابتدأ","اثر","اجل","احد","اخرى","اخلولق","اذا","اربعة","ارتدّ","استحال","اطار","اعادة","اعلنت","اف","اكثر","اكد","الألاء","الألى","الا","الاخيرة","الان","الاول","الاولى","التى","التي","الثاني","الثانية","الذاتي","الذى","الذي","الذين","السابق","الف","اللائي","اللاتي","اللتان","اللتيا","اللتين","اللذان","اللذين","اللواتي","الماضي","المقبل","الوقت","الى","اليوم","اما","امام","امس","ان","انبرى","انقلب","انه","انها","او","اول","اي","ايار","ايام","ايضا","ب","بات","باسم","بان","بخٍ","برس","بسبب","بسّ","بشكل","بضع","بطآن","بعد","بعض","بك","بكم","بكما","بكن","بل","بلى","بما","بماذا","بمن","بن","بنا","به","بها","بي","بيد","بين","بَسْ","بَلْهَ","بِئْسَ","تانِ","تانِك","تبدّل","تجاه","تحوّل","تلقاء","تلك","تلكم","تلكما","تم","تينك","تَيْنِ","تِه","تِي","ثلاثة","ثم","ثمّ","ثمّة","ثُمَّ","جعل","جلل","جميع","جير","حار","حاشا","حاليا","حاي","حتى","حرى","حسب","حم","حوالى","حول","حيث","حيثما","حين","حيَّ","حَبَّذَا","حَتَّى","حَذارِ","خلا","خلال","دون","دونك","ذا","ذات","ذاك","ذانك","ذانِ","ذلك","ذلكم","ذلكما","ذلكن","ذو","ذوا","ذواتا","ذواتي","ذيت","ذينك","ذَيْنِ","ذِه","ذِي","راح","رجع","رويدك","ريث","رُبَّ","زيارة","سبحان","سرعان","سنة","سنوات","سوف","سوى","سَاءَ","سَاءَمَا","شبه","شخصا","شرع","شَتَّانَ","صار","صباح","صفر","صهٍ","صهْ","ضمن","طاق","طالما","طفق","طَق","ظلّ","عاد","عام","عاما","عامة","عدا","عدة","عدد","عدم","عسى","عشر","عشرة","علق","على","عليك","عليه","عليها","علًّ","عن","عند","عندما","عوض","عين","عَدَسْ","عَمَّا","غدا","غير","ـ","ف","فان","فلان","فو","فى","في","فيم","فيما","فيه","فيها","قال","قام","قبل","قد","قطّ","قلما","قوة","كأنّما","كأين","كأيّ","كأيّن","كاد","كان","كانت","كذا","كذلك","كرب","كل","كلا","كلاهما","كلتا","كلم","كليكما","كليهما","كلّما","كلَّا","كم","كما","كي","كيت","كيف","كيفما","كَأَنَّ","كِخ","لئن","لا","لات","لاسيما","لدن","لدى","لعمر","لقاء","لك","لكم","لكما","لكن","لكنَّما","لكي","لكيلا","للامم","لم","لما","لمّا","لن","لنا","له","لها","لو","لوكالة","لولا","لوما","لي","لَسْتَ","لَسْتُ","لَسْتُم","لَسْتُمَا","لَسْتُنَّ","لَسْتِ","لَسْنَ","لَعَلَّ","لَكِنَّ","لَيْتَ","لَيْسَ","لَيْسَا","لَيْسَتَا","لَيْسَتْ","لَيْسُوا","لَِسْنَا","ما","ماانفك","مابرح","مادام","ماذا","مازال","مافتئ","مايو","متى","مثل","مذ","مساء","معاذ","مقابل","مكانكم","مكانكما","مكانكنّ","مكانَك","مليار","مليون","مما","ممن","من","منذ","منها","مه","مهما","مَنْ","مِن","نحن","نحو","نعم","نفس","نفسه","نهاية","نَخْ","نِعِمّا","نِعْمَ","ها","هاؤم","هاكَ","هاهنا","هبّ","هذا","هذه","هكذا","هل","هلمَّ","هلّا","هم","هما","هن","هنا","هناك","هنالك","هو","هي","هيا","هيت","هيّا","هَؤلاء","هَاتانِ","هَاتَيْنِ","هَاتِه","هَاتِي","هَجْ","هَذا","هَذانِ","هَذَيْنِ","هَذِه","هَذِي","هَيْهَاتَ","و","و6","وا","واحد","واضاف","واضافت","واكد","وان","واهاً","واوضح","وراءَك","وفي","وقال","وقالت","وقد","وقف","وكان","وكانت","ولا","ولم","ومن","مَن","وهو","وهي","ويكأنّ","وَيْ","وُشْكَانََ","يكون","يمكن","يوم","ّأيّان"}
def remove_stop_words(tweet):
    zen = TextBlob(tweet)
    words = zen.words
    return " ".join([w for w in words if not w in ar_stops and not w in stop_words and len(w) >= 2])


In [58]:

def clean_up(tweet):
    tweet = remove_diacritics(tweet)
    tweet = remove_links_foreign_words(tweet)
    tweet = remove_stop_words(tweet)
    tweet = remove_emojis_unrelated_chars(tweet)
    return tweet
 


In [59]:
dirty_tweets["Tweet"] = dirty_tweets["Tweet"].apply(lambda tweet: clean_up(tweet))


In [60]:
display(dirty_tweets.head())
display(dirty_tweets.tail())


,ID,Tweet,Timestamp,Likes,Retweets,Length
0,1497300372569739264,قبيل الاحتفال بـ اليوم الدولي لمراة تستعد كوكبة الفنانات الاستثناءيات وفرق موسيقية بقيادات نساءية لابهار زوار اكسبو 2020 دبي عينك علا اكسبو expo2020 Expo2020 3yfbIstF6,2022-02-25 20:00:01+00:00,1,0,198
1,1497299670522994691,JIMY 4040 لحظات نزول دابانغ سلمان خان علا ارض المسرح اكسبو 2020 يقف علا مجسم لجناحي صقر والتصفيق مستمر الجمهور Expo2020 expo2020 TBM7OL4,2022-02-25 19:57:14+00:00,2,0,207
2,1497297255245754371,ليلة ميزة بانتظارنا expo2020 ٢٦ الحانها شرقية نجومها استثناءين Expo2020 Expo2020 اكسبو2020,2022-02-25 19:47:38+00:00,0,0,232
3,1497295489481510914,اصدق حفلة احلا فستان مستان انا بختار Expo2020 واذا حفلة فبختار حفلة حفلات نانسي كلون حلوين يعني,2022-02-25 19:40:37+00:00,3,1,168
4,1497290589800480779,JIMY 4040 جنون بعده جنون دابانغ سلمان خان يستعرض حشد كبير معجبينه اكسبو 2020 Expo2020 expo2020 5CSoLlsQW0,2022-02-25 19:21:09+00:00,5,2,178


,ID,Tweet,Timestamp,Likes,Retweets,Length
7977,1509875797544378374,عمان المجد والتاريخ والتقدم العلمي وحصول السلطنة علا الجاءزة الذهبية اكسبو 2020 دبي tu9,2022-04-01 12:50:16+00:00,0,0,112
7978,1509875571114795011,جناح السعودية يختم فعالياته معرض اكسبو 2020 دبي بارقام قياسية ونجاحات استثناءية EA7,2022-04-01 12:49:22+00:00,0,0,109
7979,1509875169782865932,اكسبو 2020 دبي يختم اعماله باهم المءتمرات العالمية ورحلة فلسطين اكسبو 2020 المزيد حلقة جديدة برنامج اكسبو في اسبوع لين خضير jzjSXCQ06 Fu96hx1Q2,2022-04-01 12:47:47+00:00,7,0,196
7980,1509874863539998768,نظمت الادارة العامة لامن المطارات ومبادرة الروح الايجابية الادارة العامة لاسعاد المجتمع شرطة دبي مسيرة رحلة الخمسين احتفاء بمرور 50 علا تاسيس الدولة ونجاح فعاليات اكسبو 2020 دبي y3EYrKi73,2022-04-01 12:46:34+00:00,1,0,221
7981,1509874781964980273,ختامها مسك جناح الملكة اكسبو ٢٠٢٠ دبي يحق اعلا نسب زيارات تاريخ معارض اكسبو الدولية علا الاطلاق بنحو ملاين زاءر مختلف الجنسيات,2022-04-01 12:46:14+00:00,1,0,169
